In [9]:
# iterate through all the json files in the data directory, create a list of playlist names and pid's
import os, re, json
import numpy as np
import pandas as pd
import ijson

playlist_name = {}


for file in os.listdir('/Volumes/TOSHIBAEXT/spotify_million_playlist_dataset/data/'):
    print(file)
    if file.endswith('.json'):
        with open(os.path.join('/Volumes/TOSHIBAEXT/spotify_million_playlist_dataset/data/', file), "rb") as f:
            # use ijson to get playlist names
                for playlist in ijson.items(f, "playlists.item"):
                    if playlist['name'] not in playlist_name:
                        playlist_name[playlist['name']] = []
                    for track in playlist['tracks']:
                        if track['track_uri'][14:] not in playlist_name[playlist['name']]:
                            playlist_name[playlist['name']].append(track['track_uri'][14:])


mpd.slice.549000-549999.json
mpd.slice.613000-613999.json
mpd.slice.115000-115999.json
mpd.slice.778000-778999.json
mpd.slice.290000-290999.json
mpd.slice.596000-596999.json
mpd.slice.324000-324999.json
mpd.slice.422000-422999.json
mpd.slice.974000-974999.json
mpd.slice.679000-679999.json
mpd.slice.7000-7999.json
mpd.slice.391000-391999.json
mpd.slice.497000-497999.json
mpd.slice.225000-225999.json
mpd.slice.523000-523999.json
mpd.slice.875000-875999.json
mpd.slice.448000-448999.json
mpd.slice.712000-712999.json
mpd.slice.193000-193999.json
mpd.slice.38000-38999.json
mpd.slice.695000-695999.json
mpd.slice.899000-899999.json
mpd.slice.721000-721999.json
mpd.slice.510000-510999.json
mpd.slice.846000-846999.json
mpd.slice.216000-216999.json
mpd.slice.411000-411999.json
mpd.slice.947000-947999.json
mpd.slice.317000-317999.json
mpd.slice.22000-22999.json
mpd.slice.794000-794999.json
mpd.slice.126000-126999.json
mpd.slice.998000-998999.json
mpd.slice.620000-620999.json
mpd.slice.3000-3999.js

OSError: [Errno 5] Input/output error

In [12]:
with open('/Volumes/TOSHIBAEXT/spotify_million_playlist_dataset/data/mpd.slice.302000-302999.json', "rb") as f:
        for playlist in ijson.items(f, "playlists.item"):
            if playlist['name'] not in playlist_name:
                playlist_name[playlist['name']] = []
            for track in playlist['tracks']:
                if track['track_uri'][14:] not in playlist_name[playlist['name']]:
                    playlist_name[playlist['name']].append(track['track_uri'][14:])


with open('/Volumes/TOSHIBAEXT/spotify_million_playlist_dataset/data/mpd.slice.404000-404999.json', "rb") as f:
        for playlist in ijson.items(f, "playlists.item"):
            if playlist['name'] not in playlist_name:
                playlist_name[playlist['name']] = []
            for track in playlist['tracks']:
                if track['track_uri'][14:] not in playlist_name[playlist['name']]:
                    playlist_name[playlist['name']].append(track['track_uri'][14:])


# convert the dictionary to a dataframe
df = pd.DataFrame(list(playlist_name.items()),columns = ['playlist_name','track_uri'])

# convert the dataframe to a csv file and save 
# df.to_csv('playlist_name.csv', index=False)

In [125]:
import os, re, json
import numpy as np
import pandas as pd
import ijson

from cleantext import clean
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import emoji

# read in the csv file
df = pd.read_csv('playlist_name.csv')

def clean_text(text):
    text = clean(text, no_line_breaks=True, no_urls=True, no_punct=True, no_digits=True, replace_with_punct="", replace_with_digit="")
    text = emoji.replace_emoji(text, replace='')
    text = text.strip()
    text = text.replace('~', '')
    text = text.replace('_', '')
    text = text.replace('$', '')
    text = text.replace('^', '')
    # remove all mathemtical symbols, including =, +, -, *, /, <, >, %
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text_tokens = word_tokenize(text)
    cleaned_tokens = [w for w in text_tokens if not w in stopwords.words('english')]
    text = (" ").join(cleaned_tokens)
    if len(text) < 3:
        return ""
    return text

# rename playlist_names with cleaned text, remove empty rows
df['playlist_name'] = df['playlist_name'].apply(clean_text)
df = df[df['playlist_name'] != ""]

# save to a new csv file
df.to_csv('playlist_name_cleaned.csv', index=False)


In [87]:
import numpy as np
import pandas as pd

# open the cleaned csv file
named_df = pd.read_csv('playlist_name_cleaned.csv')

In [16]:
import os

os.environ['SPOTIPY_CLIENT_ID']='3e8627e438fa4781abf064e0d3bc7b6f'
os.environ['SPOTIPY_CLIENT_SECRET']='0ac3077f50b54e248fc32c64c8fc9e13'

In [13]:
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

playlist_dict = named_df.set_index('playlist_name').T.to_dict('list')

data = [word_tokenize(str(playlist)) for playlist in playlist_dict.keys()]

# Create CBOW model
model1 = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100, window = 5)

# Print top 5 most similar words to 'love'
print(model1.wv.most_similar('love', topn = 5))


/var/folders/v1/ljzyxbgj1xx_syzq56dgnsk00000gn/T/ipykernel_4853/676878940.py:5: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  playlist_dict = named_df.set_index('playlist_name').T.to_dict('list')


[('pixar', 0.3581070005893707), ('tururu', 0.3571394979953766), ('mountain', 0.3368164300918579), ('safari', 0.3360881507396698), ('paint', 0.33370938897132874)]


In [88]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# initialize spotipy
client_credentials_manager = SpotifyClientCredentials()
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlist_dict = named_df.set_index('playlist_name').T.to_dict('list')

for key in list(playlist_dict.keys()):
    tracklist = eval(playlist_dict[key][0])
    playlist_dict[key] = tracklist
    if len(tracklist) > 50: 
        del playlist_dict[key]

# for each playlist, replace the track_uri with the track name
for key in playlist_dict.keys():
    tracks = spotify.tracks(playlist_dict[key])
    playlist_dict[key] = [track['name'] for track in tracks['tracks']]


/var/folders/v1/ljzyxbgj1xx_syzq56dgnsk00000gn/T/ipykernel_4853/1406795209.py:8: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  playlist_dict = named_df.set_index('playlist_name').T.to_dict('list')


In [19]:
# turn the dictionary into a dataframe and save to a csv file
df = pd.DataFrame.from_dict(playlist_dict, orient='index')
df.to_csv('playlist_track.csv', index=False)

In [86]:
# turn named_df into a dictionary
named_dict = named_df.set_index('playlist_name').T.to_dict('list')
new_dict = {}

# # for each playlist, replace the track_uri with the track name
count = 0
for key in named_dict.keys():
    count +=1 
    print(count)
    if len(eval(named_dict[key][0])) < 50:
        if count == 2623:
            tracks = ["spotify:track:"+ track for track in eval(named_dict[key][0])]
            tracks = spotify.tracks(tracks)
            tracks = tracks['tracks']
            print(tracks)
            tracks = [track['name'] for track in tracks]
            new_dict[key] = tracks


/var/folders/v1/ljzyxbgj1xx_syzq56dgnsk00000gn/T/ipykernel_4853/2443393988.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  named_dict = named_df.set_index('playlist_name').T.to_dict('list')


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: 